In [2]:
import pandas as pd
import glob
import numpy as np

In [34]:
#Sjekker hva som ligger i mappen:

#paths =  spark.read.path('/produkt/kostra/regnskap/demo/utpakket/*')
#paths.show(100, False)

paths = glob.glob("/produkt/kostra/regnskap/demo/utpakket/*")
paths

#På linux (prod) blir dette:
#paths = glob.glob("/ssb/stamme03/komakro/komm_aarsregn_ubearb/arkiv/primk_regnskap/*")
#paths

[]

In [ ]:
#Henter filen, sjekker innhold
regn_2020 = pd.read_table("/ssb/stamme03/komakro/komm_aarsregn_ubearb/arkiv/primk_regnskap/g2020.txt")
regn_2020.info()
regn_2020.head(5)

In [ ]:
#Fil med faste posisjoner. Må benytte fwf (fixed width) og definere posisjonene
regn_2020 = pd.read_fwf("/ssb/stamme03/komakro/komm_aarsregn_ubearb/arkiv/primk_regnskap/g2020.txt",
    names=["skjema", "periode", "kvartal", "komnr", "orgnr", "foretaksnr" ,"kontoklasse", "funksjon", "art", "belop"],
    dtype=object,
    na_values={"."," ."},
    widths=[2, 4, 1, 6, 9, 9, 1, 4, 3, 9]
)
regn_2020.info()
regn_2020.head(2)

In [ ]:
len(regn_2020)

In [ ]:
regn_2020.dtypes

In [ ]:
#Retter datatyde
regn_2020['belop'] = regn_2020['belop'].astype("int")
regn_2020['kvartal'] = regn_2020['kvartal'].astype("str")
regn_2020['orgnr'] = regn_2020['orgnr'].astype("str")
regn_2020['foretaksnr'] = regn_2020['foretaksnr'].astype("str")
regn_2020.convert_dtypes().info()

In [ ]:
#Lagrer til Parquet
regn_2020.to_parquet("../../data/arbeid_mtk.parquet")

In [ ]:
#Henter inn parquet-fil
regn = pd.read_parquet("../../data/arbeid_mtk.parquet")

In [ ]:
regn.info()

In [ ]:
#Ajekker om det er duplikater
regn.duplicated()
regn[regn.duplicated()]

In [ ]:
#Filtrerer ut Oslo
regn[regn['komnr'] == '0301']

In [ ]:
#Filtrerer ut Oslo og driftsregnskapet
regn[(regn['komnr'].str[:2] == '03') & (regn['kontoklasse'] == '1')]

In [ ]:
Antall unike verdier i komnr
regn['komnr'].value_counts()

In [ ]:
Kjører en spørring 
x = regn.query("funksjon == '234' & art == '185'")
x

In [ ]:
#Sjekker om de to første posisjonene i komnr er siffer
regn["komnr"].astype(str).str[2].str.isdigit()

In [ ]:
#Filtrerer i regnskapsfilen
regn[(regn["funksjon"] == "234") & (regn["art"] == "185")]

In [ ]:
#Lager et objekt med spesifikasjoner for å filtrere i hovedfilen
maske = (regn["kontoklasse"] == "1") & (regn["funksjon"] == "234") & (regn["art"] == "185")
regn[maske]